# Exctracting model and dataset preprocessing

**ATTENTION:**

Notebook language: **R**

A model from a paper [EXPLAINABLE EXPECTED GOAL MODELS FOR PERFORMANCE
ANALYSIS IN FOOTBALL ANALYTICS](https://arxiv.org/pdf/2206.07212.pdf) will be used for further research. It is a `ranger` model trained on oversampled dataset.

## Preparing preprocessed dataset

In [1]:
df <- read.csv('./data/raw_data.csv')
df <- df[,-1]
head(df)

,league,id,minute,result,X,Y,player,h_a,player_id,situation,season,shotType,match_id,home_team,away_team,home_goals,away_goals,date,player_assisted,lastAction
,<fct>,<int>,<int>,<fct>,<dbl>,<dbl>,<fct>,<fct>,<int>,<fct>,<int>,<fct>,<int>,<fct>,<fct>,<int>,<int>,<fct>,<fct>,<fct>
1,Ligue_1,425095,7,MissedShots,0.964,0.654,Myron Boadu,h,9612,OpenPlay,2021,LeftFoot,17822,Monaco,Nantes,1,1,2021-08-06 19:00:00,NA,BallRecovery
2,Ligue_1,425098,13,Goal,0.925,0.431,Gelson Martins,h,7012,OpenPlay,2021,RightFoot,17822,Monaco,Nantes,1,1,2021-08-06 19:00:00,Caio Henrique,Throughball
3,Ligue_1,425100,24,BlockedShot,0.785,0.388,Kevin Volland,h,83,OpenPlay,2021,LeftFoot,17822,Monaco,Nantes,1,1,2021-08-06 19:00:00,NA,None
4,Ligue_1,425101,24,MissedShots,0.761,0.525,Jean Lucas,h,7687,OpenPlay,2021,RightFoot,17822,Monaco,Nantes,1,1,2021-08-06 19:00:00,NA,Rebound
5,Ligue_1,425102,30,MissedShots,0.936,0.415,Kevin Volland,h,83,FromCorner,2021,Head,17822,Monaco,Nantes,1,1,2021-08-06 19:00:00,Jean Lucas,Aerial
6,Ligue_1,425104,42,MissedShots,0.751,0.511,Aurelien Tchouameni,h,6560,OpenPlay,2021,RightFoot,17822,Monaco,Nantes,1,1,2021-08-06 19:00:00,Caio Henrique,Pass


Remove `OwnGoal` - it is also a first preprocessing step 

In [3]:
library(dplyr)

In [4]:
df <- df %>% filter(result != "OwnGoal")
head(df)

,league,id,minute,result,X,Y,player,h_a,player_id,situation,season,shotType,match_id,home_team,away_team,home_goals,away_goals,date,player_assisted,lastAction
,<fct>,<int>,<int>,<fct>,<dbl>,<dbl>,<fct>,<fct>,<int>,<fct>,<int>,<fct>,<int>,<fct>,<fct>,<int>,<int>,<fct>,<fct>,<fct>
1,Ligue_1,425095,7,MissedShots,0.964,0.654,Myron Boadu,h,9612,OpenPlay,2021,LeftFoot,17822,Monaco,Nantes,1,1,2021-08-06 19:00:00,NA,BallRecovery
2,Ligue_1,425098,13,Goal,0.925,0.431,Gelson Martins,h,7012,OpenPlay,2021,RightFoot,17822,Monaco,Nantes,1,1,2021-08-06 19:00:00,Caio Henrique,Throughball
3,Ligue_1,425100,24,BlockedShot,0.785,0.388,Kevin Volland,h,83,OpenPlay,2021,LeftFoot,17822,Monaco,Nantes,1,1,2021-08-06 19:00:00,NA,None
4,Ligue_1,425101,24,MissedShots,0.761,0.525,Jean Lucas,h,7687,OpenPlay,2021,RightFoot,17822,Monaco,Nantes,1,1,2021-08-06 19:00:00,NA,Rebound
5,Ligue_1,425102,30,MissedShots,0.936,0.415,Kevin Volland,h,83,FromCorner,2021,Head,17822,Monaco,Nantes,1,1,2021-08-06 19:00:00,Jean Lucas,Aerial
6,Ligue_1,425104,42,MissedShots,0.751,0.511,Aurelien Tchouameni,h,6560,OpenPlay,2021,RightFoot,17822,Monaco,Nantes,1,1,2021-08-06 19:00:00,Caio Henrique,Pass


Update `raw_data.csv` to avoid conflicts in future

In [5]:
write.csv(df, './data/raw_data.csv')

### Preprocessing

In [6]:
source('./scripts/preprocess.R') # preprocessing steps from the paper

In [7]:
df_preprocessed <- preprocess(df)
head(df_preprocessed)

,status,minute,h_a,situation,shotType,lastAction,distanceToGoal,angleToGoal
,<dbl>,<dbl>,<fct>,<fct>,<fct>,<fct>,<dbl>,<dbl>
1,0,7,h,OpenPlay,LeftFoot,BallRecovery,12.554569,10.86049
2,1,13,h,OpenPlay,RightFoot,Throughball,8.497323,44.42738
3,0,24,h,OpenPlay,LeftFoot,None,23.388803,17.20585
4,0,24,h,OpenPlay,RightFoot,Rebound,25.298204,16.33905
5,0,30,h,FromCorner,Head,Aerial,7.967234,44.48587
6,0,42,h,OpenPlay,RightFoot,Pass,26.241467,15.82464


In [8]:
factor_cols <- unlist(lapply(df_preprocessed, is.factor))
factor_cols <- names(factor_cols[factor_cols == TRUE])

for (col in factor_cols){
    df_preprocessed[,col] <- as.integer(df_preprocessed[,col])
}

In [9]:
head(df_preprocessed)

,status,minute,h_a,situation,shotType,lastAction,distanceToGoal,angleToGoal
,<dbl>,<dbl>,<int>,<int>,<int>,<int>,<dbl>,<dbl>
1,0,7,2,3,2,2,12.554569,10.86049
2,1,13,2,3,4,39,8.497323,44.42738
3,0,24,2,3,2,24,23.388803,17.20585
4,0,24,2,3,4,29,25.298204,16.33905
5,0,30,2,2,1,1,7.967234,44.48587
6,0,42,2,3,4,27,26.241467,15.82464


In [10]:
write.csv(df_preprocessed, './data/data_preprocessed.csv')

## Extracting model to `Python` format

In [ ]:
model <- readRDS('./model/model.RDS')

In [ ]:
# install.packages("reticulate")

In [ ]:
library(reticulate)

In [ ]:
py_save_object(model$forest, './model/model-imported.pickle')